# Genetic Algorithm

**Optimization (Maximization) Problem:**

Objective function:  
  f(x) = 2x^2 - 3x + 10 \, \( 0 < x <= 20 \).

- Use roulette wheel selection.
- Perform two-point crossover for 2 generations.
- Use 6-bit chromosome fabrication.
- Perform flipping the bits mutation.
- Initial Individuals: 2, 8, 5, 11.


In [1]:
import random

def f(x):
    return 2 * x**2 - 3 * x + 10

def decimal_to_binary(n):
    return format(n, '06b')

def binary_to_decimal(b):
    return int(b, 2)

def calculate_fitness(population):
    return [f(binary_to_decimal(chromosome)) for chromosome in population]

def roulette_wheel_selection(population, fitness):
    total_fitness = sum(fitness)
    probabilities = [fit / total_fitness for fit in fitness]
    cumulative_probs = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    
    selected = []
    for _ in range(len(population)):  # Select as many as population size
        r = random.random()
        for i, cp in enumerate(cumulative_probs):
            if r <= cp:
                selected.append(population[i])
                break
    return selected

def two_point_crossover(parent1, parent2):
    p1, p2 = sorted(random.sample(range(1, 6), 2)) 
    child1 = parent1[:p1] + parent2[p1:p2] + parent1[p2:]
    child2 = parent2[:p1] + parent1[p1:p2] + parent2[p2:]
    return child1, child2

def bit_flip_mutation(chromosome, mutation_rate=0.1):
    mutated = ""
    for bit in chromosome:
        if random.random() < mutation_rate:
            mutated += '0' if bit == '1' else '1'  # Flip bit
        else:
            mutated += bit
    return mutated

def genetic_algorithm(generations, mutation_rate=0.1):
    # Initialize population
    initial_values = [2, 8, 5, 11]
    population = [decimal_to_binary(x) for x in initial_values]

    for generation in range(generations):
        print(f"\nGeneration {generation + 1}")
        #Calculate fitness
        fitness = calculate_fitness(population)
        print("Population:", population)
        print("Fitness:", fitness)
        #Selection
        parents = roulette_wheel_selection(population, fitness)
        #Crossover
        offspring = []
        for i in range(0, len(parents), 2):
            child1, child2 = two_point_crossover(parents[i], parents[(i+1) % len(parents)])
            offspring.extend([child1, child2])
        #Mutation
        offspring = [bit_flip_mutation(child, mutation_rate) for child in offspring]

        offspring = [decimal_to_binary(min(max(1, binary_to_decimal(child)), 20)) for child in offspring]
        
        # New population
        population = offspring

    final_fitness = calculate_fitness(population)
    best_index = final_fitness.index(max(final_fitness))
    best_solution = binary_to_decimal(population[best_index])
    
    print("\nBest solution:", best_solution)
    print("Maximum value:", f(best_solution))

genetic_algorithm(2)


Generation 1
Population: ['000010', '001000', '000101', '001011']
Fitness: [12, 114, 45, 219]

Generation 2
Population: ['000110', '010100', '010100', '000001']
Fitness: [64, 750, 750, 9]

Best solution: 20
Maximum value: 750


**Using libraries**


In [2]:
import numpy as np
import pygad

def f(x):
    return 2 * x**2 - 3 * x + 10

def binary_to_decimal(binary_array):
    binary_str = ''.join(str(int(bit)) for bit in binary_array)  
    return int(binary_str, 2)

def fitness_function(ga_instance, solution, solution_idx):
    x = binary_to_decimal(solution)  
    if 0 < x <= 20:  
        return f(x)  
    return -1  

initial_population = np.array([
    list(map(int, format(2, "06b"))),  
    list(map(int, format(8, "06b"))),  
    list(map(int, format(5, "06b"))),  
    list(map(int, format(11, "06b")))  
])

ga_instance = pygad.GA(
    num_generations=2,  
    num_parents_mating=2,  
    fitness_func=fitness_function,  
    sol_per_pop=4,  
    num_genes=6,  
    initial_population=initial_population,  
    gene_space=[0, 1],  
    parent_selection_type="rws",  
    crossover_type="two_points",  
    mutation_type="random",  
    mutation_percent_genes=50  
)

ga_instance.run()

solution, solution_fitness, _ = ga_instance.best_solution()

best = binary_to_decimal(solution)

print("Best Solution:", best)  
print("Maximum Value:", f(best))  

Best Solution: 11
Maximum Value: 219
